# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\norma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\norma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\norma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("etl", engine)
#df = df.head(100)
X = df.message
Y = df.iloc[:, 4:] #[['related', 'offer','aid_related']]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [27]:
def tokenize(text):
    text = text.lower()
    text = re.sub('[^A-Za-z0-9]', ' ', text)
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        #print(clean_tok)
        clean_tokens.append(clean_tok)
    clean_tokens = [w for w in tokens if w not in stopwords.words('english')]

    return clean_tokens
# test function
tokenize("Hello, I have spoken to you and your friends.")

['hello', 'spoken', 'friends']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
    ('vect' , CountVectorizer(tokenizer=tokenize, ngram_range =(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [29]:
#split data:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train pipeline
pipeline.fit(X = X_train, y= y_train)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [30]:
# make prediction for test set
y_pred = pipeline.predict(X_test)

# generate classification report
from sklearn.metrics import classification_report

def make_report(y_true, y_pred):
    """
    Generate a performance report for a multiclass prediction
    INPUT:
        y_true - true labels
        y_pred - predicted labels

    """
    for i, col in enumerate(list(y_test.columns)):
        print(col)
        report = classification_report(y_true[col], y_pred[:,i], output_dict=False, zero_division=0)
        print(report)


make_report(y_test, y_pred)

related
              precision    recall  f1-score   support

           0       0.70      0.43      0.53      1495
           1       0.85      0.94      0.89      5018
           2       0.23      0.39      0.29        41

    accuracy                           0.82      6554
   macro avg       0.59      0.59      0.57      6554
weighted avg       0.81      0.82      0.81      6554

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5387
           1       0.86      0.50      0.63      1167

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.89      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6533
           1       0.00      0.00      0.00        21

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

### 6. Improve your model
Use grid search to find better parameters. 

In [32]:
from sklearn.model_selection import GridSearchCV
parameters =  {
              'clf__estimator__n_estimators': [50, 100]
}
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)
cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
y_pred_cv = cv.predict(X_test)
make_report(y_test, y_pred_cv)

related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.76      1.00      0.86        19

    accuracy                           0.76        25
   macro avg       0.38      0.50      0.43        25
weighted avg       0.58      0.76      0.66        25

request
              precision    recall  f1-score   support

           0       0.73      0.80      0.76        10
           1       0.86      0.80      0.83        15

    accuracy                           0.80        25
   macro avg       0.79      0.80      0.79        25
weighted avg       0.81      0.80      0.80        25

offer
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        24
           1       0.00      0.00      0.00         1

    accuracy                           0.96        25
   macro avg       0.48      0.50      0.49        25
weighted avg       0.92      0.96      0.94        2

C:\Users\norma\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# trying to improve the model by changing the algorithm and adding 
from sklearn.ensemble import AdaBoostClassifier

pipeline_imp = Pipeline([
    ('vect' , CountVectorizer(tokenizer=tokenize, ngram_range =(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    
])

parameters_imp =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10, 50], 
              'clf__estimator__learning_rate':[1.0, 2.0]} 
cv_imp = GridSearchCV(pipeline_imp, param_grid=parameters_imp)
cv_imp.fit(X_train, y_train)

y_pred_imp = cv_imp.fit(X_test)
make_report(y_test, y_pred_imp)

C:\Users\norma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: y cannot be None

  FitFailedWarning)
C:\Users\norma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: y cannot be None

  FitFailedWarning)
C:\Users\norma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: y cannot be None

  FitFailedWarning)
C:\Users\norma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set 

ValueError: y cannot be None

### 9. Export your model as a pickle file

In [57]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.